In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [3]:
description=pd.read_csv('report-2L-QC-1103.pg_matrix.tsv',sep='\t',index_col='Protein.Group')

In [5]:
drop_info=['Protein.Ids','Protein.Names','Genes','First.Protein.Description']
df=description.drop(drop_info,axis=1)

In [7]:
df=df.fillna(0)

In [9]:
#Adjust column names
df_column=[]
for i in range(df.shape[1]):
    if 'QC' in df.columns[i].split('\\')[4]:
        break
    if i%2==0:
        df_column.append('2L_'+str(int((i/2)+1))+'_1')
    if i%2==1:
        df_column.append('2L_'+str(int((i+1)/2))+'_2')
for i in range(df.shape[1]):
    if 'QC' in df.columns[i].split('\\')[4]:
        df_column.append('QC_'+df.columns[i].split('\\')[4].split('_')[4].split('.')[0])

In [11]:
df.columns=df_column

In [13]:
#Filter contaminates
keratin=[]
for i in range(description.shape[0]):
    if 'HUMAN' in description.loc[description.index[i],'Protein.Names']:
        keratin.append(description.index[i])

In [15]:
trypsin_bsa=['P00761','P02769']
con=keratin+trypsin_bsa

In [17]:
df_filtered_con=df.drop(con,axis=0)

In [19]:
QC_sample=[]
for i in range(df.shape[1]):
    if 'QC' in df.columns[i]:
        QC_sample.append(df.columns[i])
len(QC_sample)

25

In [21]:
import random

In [23]:
#Filter one QC sample to ensure the total number of QC samples is plural
drop_QC=random.randint(0, 24)

In [25]:
QC_sample[21]

'QC_2309'

In [27]:
QC_sample.remove('QC_2309')

In [29]:
df_filtered_1QC=df_filtered_con.drop('QC_2309',axis=1)

In [31]:
#Filter out proteins with too many NA values
drop_na=[]
for i in range(df_filtered_1QC.shape[0]):
    count_sample=0
    count_qc=0
    for j in range(df_filtered_1QC.shape[1]):
        if df_filtered_1QC.iloc[i,j]==0 and '2L' in df_filtered_1QC.columns[j]:
            count_sample+=1
        if df_filtered_1QC.iloc[i,j]==0 and 'QC' in df_filtered_1QC.columns[j]:
            count_qc+=1
    if count_sample>18 or count_qc>12:
        drop_na.append(df_filtered_1QC.index[i])
len(drop_na)

524

In [33]:
df_filtered_na=df_filtered_1QC.drop(drop_na)

In [35]:
sample_columns=[]
qc_columns=[]
for i in range(df_filtered_na.shape[1]):
    if 'QC' not in df_filtered_na.columns[i]:
        sample_columns.append(df_filtered_na.columns[i])
    if 'QC' in df_filtered_na.columns[i]:
        qc_columns.append(df_filtered_na.columns[i])

In [37]:
df_sample=df_filtered_na[sample_columns]
df_qc=df_filtered_na[qc_columns]

In [49]:
embryo_2L={}
for col in df_sample.columns:
    if col.split('_')[0]=='2L':
        label=col.split('_')[1]
        if label in embryo_2L:
            embryo_2L[label].append(col)
        else:
            embryo_2L[label]=[col]

In [51]:
from numpy import std,mean

In [53]:
df_sample_cv=df_sample.copy()
df_sample_cv=df_sample_cv.drop(df_sample.columns,axis=1)
for embryo in embryo_2L:
    df_sample_cv['cv_sample_'+embryo] =std(df_sample[embryo_2L[embryo]],ddof=0,axis=1)/mean(df_sample[embryo_2L[embryo]],axis=1)